In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import h5py
from matplotlib import pyplot as plt

In [2]:
transcripts = pd.read_csv('transcript_canonical_test.csv', index_col=0)
len(transcripts)

16865

In [3]:
transcripts2 = pd.read_csv('transcript_canonical_longest_test.csv', index_col=0)
len(transcripts2)

1275

In [4]:
transcripts = pd.concat([transcripts, transcripts2])
len(transcripts)

18140

In [5]:
genes = pd.read_csv('gencode.v22.annotation.test.CDS.gn.csv', index_col=0)
genes.iloc[:5]

,chr,type,start,end,strand,gene,genename,transcriptID
692,chr1,transcript,924880,939291,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
693,chr1,exon,924880,924948,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
694,chr1,exon,925922,926013,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
695,chr1,CDS,925942,926013,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
696,chr1,start_codon,925942,925944,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""


In [6]:
tokeep = np.where(genes['type'] != 'transcript')[0]
genes = genes.iloc[tokeep]
genes.iloc[:5]

,chr,type,start,end,strand,gene,genename,transcriptID
693,chr1,exon,924880,924948,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
694,chr1,exon,925922,926013,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
695,chr1,CDS,925942,926013,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
696,chr1,start_codon,925942,925944,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""
697,chr1,exon,930155,930336,+,"""ENSG00000187634.9""",SAMD11,"""ENST00000420190.4"""


In [7]:
hl = np.zeros((len(transcripts),6))
hl = pd.DataFrame(data=hl, index=transcripts.values[:,0],
                 columns=['ORF', '5UTR', '3UTR', 'intron', 'exonjunction', 'genename'])

In [8]:
genes['transcriptID'] = np.char.strip(np.array(genes['transcriptID'], dtype=str), '"')
genes['transcriptID'] = genes['transcriptID'].str.split('.').str[0].values

In [9]:
for t in tqdm(transcripts.values):
    t = t[0]
    idx = np.where(genes['transcriptID'] == t)[0]
    sub = genes.iloc[idx]
    sub = sub.sort_values(['start'])
    gn = sub['genename'].values[0]
    hl['genename'].loc[t] = gn
    
    # Check if on positive strand
    if(sub['strand'].iloc[0] == '+'):
        
        # 5' UTR length -> from first location exon to start codon
        firstexon = np.min(sub['start'].values)
        if(len(np.where(sub['type'] == 'start_codon')[0]) == 0):
            print('No start codon defined')
            print(gn)
            startcodon = sub['start'].values[np.where(sub['type'] == 'CDS')[0][0]]
            
        else:
            startcodon = sub['start'].values[np.where(sub['type'] == 'start_codon')[0][0]]
            if(len(np.where(sub['type'] == 'start_codon')[0]) > 1):
                print(gn)
        utr5 = startcodon-firstexon
        hl['5UTR'].loc[t] = utr5
        
        # ORF length -> sum length CDS
        idx_cds = np.where(sub['type'] == 'CDS')[0]
        sub_cds = sub.iloc[idx_cds]
        len_cds = np.sum(sub_cds['end']-sub_cds['start']+1)
        hl['ORF'].loc[t] = len_cds
                
        # 3' UTR length -> from location last CDS to last location exon
        lastexon = np.max(sub['end'].values)
        lastCDS = np.max(sub_cds['end'].values)
        utr3 = lastexon-lastCDS
        hl['3UTR'].loc[t] = utr3
            
    else:
        # 5' UTR length -> from last location exon to start codon
        lastexon = np.max(sub['end'].values)
        if(len(np.where(sub['type'] == 'start_codon')[0]) == 0):
            print('No start codon defined')
            print(gn)
            startcodon = np.sort(sub['end'].values[np.where(sub['type'] == 'CDS')[0]])[-1]

        else:
            startcodon = sub['end'].values[np.where(sub['type'] == 'start_codon')[0][-1]]
            if(len(np.where(sub['type'] == 'start_codon')[0]) > 1):
                print(gn)

        utr5 = lastexon-startcodon
        hl['5UTR'].loc[t] = utr5
        
        # ORF length -> sum length CDS
        idx_cds = np.where(sub['type'] == 'CDS')[0]
        sub_cds = sub.iloc[idx_cds]
        len_cds = np.sum(sub_cds['end']-sub_cds['start']+1)
        hl['ORF'].loc[t] = len_cds
        
        # 3' UTR length -> from first location exon to first CDS
        firstexon = np.min(sub['start'].values)
        firstCDS = np.min(sub_cds['start'].values)
        utr3 = firstCDS-firstexon
        hl['3UTR'].loc[t] = utr3

    # intron length -> length transcript - ORF - 3'UTR - 5' UTR
    len_tr = lastexon - firstexon + 1
    hl['intron'].loc[t] = len_tr-len_cds-utr3-utr5

    # exon junction density -> #exons/ORF * 1000
    num_exon = len(np.where(sub['type'] == 'exon')[0])
    hl['exonjunction'].loc[t] = (num_exon*1000)/len_cds


  0%|          | 0/18140 [00:00<?, ?it/s]

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

...

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['5UTR'].loc[t] = utr5
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['ORF'].loc[t] = len_cds
C:\Users\LCMMIC~1\AppData\Local\Temp/ipykernel_14944/1818864361.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hl['3UTR'].loc[t] = utr3
C:\Users\LC

KeyboardInterrupt: 

In [10]:
hl.index = hl['genename']
hl = hl.drop(columns=['genename'])
hl

,ORF,5UTR,3UTR,intron,exonjunction
genename,,,,,
ARF5,540.0,154.0,409.0,2258.0,11.111111
M6PR,831.0,3551.0,1456.0,3753.0,8.423586
ESRRA,1269.0,1602.0,775.0,7515.0,5.516154
FKBP4,1377.0,187.0,2168.0,6739.0,7.262164
CYP26B1,1536.0,204.0,2992.0,14069.0,3.906250
...,...,...,...,...,...
0.0,0.0,0.0,0.0,0.0,0.000000
0.0,0.0,0.0,0.0,0.0,0.000000
0.0,0.0,0.0,0.0,0.0,0.000000


In [11]:
hl = np.log10(hl+0.1)

In [13]:
from sklearn.preprocessing import StandardScaler

hl2 = StandardScaler().fit_transform(hl)

In [14]:
hl2 = pd.DataFrame(hl2, index=hl.index, columns=hl.columns)

In [15]:
hl2.to_csv('HL_normalized_test.csv')